In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/GRU #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [5]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        training_data_y = np.load(f)
        test_y = np.load(f)
        
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [6]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
221/221 [==============================] - 50s 206ms/step - loss: 0.6879 - accuracy: 0.5574 - auc: 0.5594 - precision: 0.5355 - recall: 0.8777 - false_negatives: 433.0000 - false_positives: 2696.0000 - true_negatives: 833.0000 - true_positives: 3108.0000 - val_loss: 0.6829 - val_accuracy: 0.5746 - val_auc: 0.5702 - val_precision: 0.5453 - val_recall: 0.8781 - val_false_negatives: 184.0000 - val_false_positives: 1105.0000 - val_true_negatives: 416.0000 - val_true_positives: 1325.0000
Epoch 2/10
221/221 [==============================] - 42s 191ms/step - loss: 0.6825 - accuracy: 0.5711 - auc: 0.5744 - precision: 0.5471 - recall: 0.8351 - false_negatives: 584.0000 - false_positives: 2448.0000 - true_negatives: 1081.0000 - true_positives: 2957.0000 - val_loss: 0.6777 - val_accuracy: 0.5815 - val_auc: 0.5906 - val_precision: 0.5522 - val_recall: 0.8443 - val_false_negatives: 235.0000 - val_false_positives: 1033.0000 - val_true_negatives: 488.0000 - val_true_positives: 1274.0000
E

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [7]:
%tensorboard --logdir logs/GRU/fit